## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움 사용

In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse # url encode
from tqdm import tqdm, tqdm_notebook

In [3]:
driver = webdriver.Chrome()
url = 'https://search.danawa.com/dsearch.php?k1=%EC%97%90%EC%96%B4%EB%A9%94%EC%9D%B4%EB%93%9C+%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%83%80%EC%9B%8C&module=goods&act=dispMain'
driver.get(url)
time.sleep(5.0)

In [8]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [9]:
prodItems = soup.select('ul.product_list>li.prod_item')

In [12]:
prodItems[0].select('p.prod_name > a')[0].text.strip()

'에어메이드 스마트타워 AMC-3501A'

In [14]:
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t','')

'핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2024년형 / [구성] 먼지비움 / 충전 / UVC LED / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 40분(최대) / 충전시간: 5시간 / 분리형(1개) / 2500mAh / [청소] LED라이트 / BLDC모터 / [부가] 디스플레이표시 / 헤파필터 / 색상: 화이트 / 무게: 2.5kg / 크기(가로x세로x깊이): 252x1253x185mm'

In [25]:
# isdecimal 숫자가아니면 False 반환
prodItems[0].select('input')[1].get('value').isdecimal()

True

##### 다나와 무선청소기 웹크롤링 수정
- 가격 외에는 안들어오도록 변환

In [26]:
## 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
          f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
            'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
            'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [35]:
def getProdItems(prodItems):
    prodData = []
    for prodItem in prodItems:
        try:
            product_name = prodItem.select('p.prod_name >a')[0].text.strip()
            product_spec = prodItem.select('div.spec_list')[0].text.strip().replace('\t','')
            if prodItem.select('input')[1].get('value').isdecimal() == True:
                product_price = prodItem.select('input')[1].get('value')
            else:
                product_price = 0
            product_reg_mon = prodItem.select('div.prod_sub_meta> dl>dd')[0].text.strip()
            prodData.append([product_name,product_spec,product_price,product_reg_mon])
        except:
            pass
    return prodData

In [36]:
# 여러페이지 검색후 크롤링하는 작업

driver = webdriver.Chrome()

# 암묵적으로 맵 자원 로드를 위해서 3초 정도 대기한다.
driver.implicitly_wait(3)

keyword = '무선청소기'
startPage = 1
totalPage = 20
# 최종 저장 리스트
prodDataTotal = []

for page in tqdm(range(startPage,totalPage+1)):
    url = getSearchPageUrl(keyword,page)
    driver.get(url)
    #페이지 로딩이 완료될 때 까지 5초 대기
    time.sleep(5)
    
    # 상품 정보 추출
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems)

    prodDataTotal += prodItemList


100%|██████████| 20/20 [03:28<00:00, 10.41s/it]


In [38]:
len(prodDataTotal)

2120

In [39]:
dfprodDataTotal = pd.DataFrame(prodDataTotal)

In [42]:
dfprodDataTotal.columns=['제품명','제품스펙','최저가','날짜']

In [43]:
dfprodDataTotal.to_excel('./data/다나와_무선청소기_결과.xlsx',index= False)

##### 크롤링 데이터 전처리

In [47]:
# 저장한 엑셀을 재로드
dfprodDanawa = pd.read_excel('./data/다나와_무선청소기_결과.xlsx')
dfprodDanawa.tail()

,제품명,제품스펙,최저가,날짜
2115,미니센 무선 소형 청소기 MNC-100 먼지통,차량용청소기 / 먼지통,7000,2024.03.
2116,프리라벨 2in1 무선 진공 청소기 LS-2000 헤파필터,차량용청소기 / 필터 / [필터] 헤파필터,3500,2024.03.
2117,상품상세설명 참조 캐치웰 텀블러 디자인 초경량 무선 핸디형 진공청소기_아이언그레이,NaN,3500,2024.03.
2118,포쉬 워시젯 V2 WJ200,침구청소기 / 핸디형 / 무선 / 사용시간: 25분(최대) / [기능] 청소모드: ...,219000,2024.02.
2119,업킵 MJ-BC779,침구청소기 / 핸디형 / 무선 / [기능] 청소모드: UV살균 / 속도조절 / 헤파...,89900,2024.02.


In [48]:
dfprodDanawa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2120 entries, 0 to 2119
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     2120 non-null   object
 1   제품스펙    2110 non-null   object
 2   최저가     2120 non-null   int64 
 3   날짜      2120 non-null   object
dtypes: int64(1), object(3)
memory usage: 66.4+ KB


In [55]:
# 결측치 검색
condition = dfprodDanawa['제품스펙'].isnull() == True
dfprodDanawa[condition]

,제품명,제품스펙,최저가,날짜
1846,스팀청소기 샤오미 무선 휴대용 자동차 진공 청소기 스마트 홈 흡입 및 블로잉 다기능...,NaN,159000,2024.03.
1861,상품상세설명 참조 캐치웰 텀블러 디자인 초경량 무선 핸디형 진공청소기_아이언그레이,NaN,3500,2024.03.
1910,스팀청소기 샤오미 무선 휴대용 자동차 진공 청소기 스마트 홈 흡입 및 블로잉 다기능...,NaN,159000,2024.03.
1925,상품상세설명 참조 캐치웰 텀블러 디자인 초경량 무선 핸디형 진공청소기_아이언그레이,NaN,3500,2024.03.
1974,스팀청소기 샤오미 무선 휴대용 자동차 진공 청소기 스마트 홈 흡입 및 블로잉 다기능...,NaN,159000,2024.03.
1989,상품상세설명 참조 캐치웰 텀블러 디자인 초경량 무선 핸디형 진공청소기_아이언그레이,NaN,3500,2024.03.
2038,스팀청소기 샤오미 무선 휴대용 자동차 진공 청소기 스마트 홈 흡입 및 블로잉 다기능...,NaN,159000,2024.03.
2053,상품상세설명 참조 캐치웰 텀블러 디자인 초경량 무선 핸디형 진공청소기_아이언그레이,NaN,3500,2024.03.
2102,스팀청소기 샤오미 무선 휴대용 자동차 진공 청소기 스마트 홈 흡입 및 블로잉 다기능...,NaN,159000,2024.03.
2117,상품상세설명 참조 캐치웰 텀블러 디자인 초경량 무선 핸디형 진공청소기_아이언그레이,NaN,3500,2024.03.


In [57]:
# 결측치가 있는 행을 삭제   *axis = 1(열), 0(행)
dfprodDanawa = dfprodDanawa.dropna(axis=0)

In [62]:
dfprodDanawa

,제품명,제품스펙,최저가,날짜
0,LG전자 오브제컬렉션 코드제로 A9S AX9884,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / 소비전력: ...,873560,2023.04.
1,삼성전자 비스포크 제트 VS20B956AX,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 220W / 2022년형 / [구성]...,446320,2022.02.
2,샤오미 미홀 M22,핸디스틱청소기 / 무선 / 흡입형 / 소비전력: 320W / 2023년형 / [구성...,151050,2023.01.
3,LG전자 오브제컬렉션 코드제로 A9S AX9604,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 250W / 소비전력: 620W / ...,715570,2023.04.
4,LG전자 오브제컬렉션 코드제로 A9S AX9988,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 280W / 소비전력: ...,1229240,2023.05.
...,...,...,...,...
2114,미니센 무선 소형 청소기 MNC-100 노즐 세트,차량용청소기 / 전용브러쉬,7000,2024.03.
2115,미니센 무선 소형 청소기 MNC-100 먼지통,차량용청소기 / 먼지통,7000,2024.03.
2116,프리라벨 2in1 무선 진공 청소기 LS-2000 헤파필터,차량용청소기 / 필터 / [필터] 헤파필터,3500,2024.03.
2118,포쉬 워시젯 V2 WJ200,침구청소기 / 핸디형 / 무선 / 사용시간: 25분(최대) / [기능] 청소모드: ...,219000,2024.02.


In [60]:
condition = dfprodDanawa['최저가'] == 0

In [63]:
dfprodDanawa = dfprodDanawa[condition == 0]

In [64]:
# 행을 삭제하면 인데스가 꼬임. 인덱스 초기화
dfprodDanawa.reset_index(drop=True, inplace=True)

In [66]:
dfprodDanawa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2110 entries, 0 to 2109
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     2110 non-null   object
 1   제품스펙    2110 non-null   object
 2   최저가     2110 non-null   int64 
 3   날짜      2110 non-null   object
dtypes: int64(1), object(3)
memory usage: 66.1+ KB


In [73]:
# 회사명, 제품명 분리 split (''.n)  n = 자를 공백의 번호
dfprodDanawa['제품명'].head()[0].split(' ',1)

['LG전자', '오브제컬렉션 코드제로 A9S AX9884']

In [75]:
# 회사명, 모델명 분리
compList = []
modelList = []
count = 0

for temp in dfprodDanawa['제품명']:
    titles = temp.split(' ',1)  # 길이가 2 배열 생성
    if (len(titles)) > 1 :
        compList.append(titles[0])
        modelList.append(titles[1])
    else:
        compList.append('')
        modelList.append(titles[0])
    count += 1
    

In [82]:
# modelList 와 compList 는 서로 같은 len 값을 가져야한다
len(modelList)
len(compList)

2110

In [157]:
# 스펙목록 데이터
specList = dfprodDanawa.loc[0,'제품스펙'].split(' / ')

In [87]:
useTime = []
suctionPow = []
for spec in specList:
    if '사용시간' in spec:
        useTime = spec
    elif '흡입력' in spec:
        suctionPow = spec
useTime.split(':')[1].strip()
suctionPow.split(':')[1].strip()

'250W'

In [171]:
# 카테고리, 사용시간, 흡입력 추출
CateList = []
useTimeList = []
suctionPowList = []

for spec in dfprodDanawa['제품스펙']:
    specList = spec.split(' / ')
    category = specList[0]
    CateList.append(category)
    useTimeVal = None
    suctionPowVal = None
    for temp in specList:
        if '사용시간' in temp.replace('[배터리] ',''):
            useTimeVal = temp
        elif '흡입력' in temp:
            suctionPowVal = temp

    if useTimeVal != None:
        useTime = useTimeVal.split(' ')[1].strip().replace('(최대)','')

    else:
        useTime =''

    if suctionPowVal != None:
        suctionPow = suctionPowVal.split(' ')[1].strip()
    else:
        suctionPow =''
    if useTime == '사용시간:':
        useTime = useTimeVal.split(' ')[2].strip().replace('(최대)','')
    useTimeList.append(useTime)
    suctionPowList.append(suctionPow)


In [170]:
useTimeList

['30분',
 '1시간',
 '',
 '30분',
 '1시간',
 '1시간',
 '30분',
 '1시간',
 '50분',
 '50분',
 '1시간30분',
 '1시간',
 '30분',
 '50분',
 '1시간',
 '12분',
 '30분',
 '1시간',
 '1시간',
 '1시간20분',
 '1시간',
 '1시간',
 '30분',
 '1시간',
 '1시간',
 '30분',
 '43분',
 '43분',
 '1시간',
 '1시간',
 '40분',
 '53분',
 '40분',
 '1시간',
 '1시간',
 '1시간',
 '1시간',
 '50분',
 '20분',
 '10~15분',
 '1시간',
 '1시간',
 '30분',
 '30분',
 '45분',
 '',
 '18분',
 '1시간',
 '1시간',
 '30분',
 '40분',
 '42분',
 '1시간',
 '50분',
 '1시간',
 '50분',
 '30분',
 '60분',
 '30분',
 '32분',
 '1시간',
 '1시간',
 '30분',
 '30분',
 '1시간40분',
 '1시간',
 '1시간',
 '30분',
 '1시간',
 '1시간20분',
 '1시간',
 '1시간',
 '30분',
 '30분',
 '30분',
 '1시간',
 '40분',
 '30분',
 '40분',
 '30분',
 '30분',
 '30분',
 '30분',
 '1시간',
 '30분',
 '1시간',
 '',
 '57분',
 '1시간',
 '1시간',
 '1시간',
 '1시간',
 '1시간',
 '1시간',
 '1시간',
 '35분',
 '20분',
 '50분',
 '40분',
 '45분',
 '45분',
 '30분',
 '40분',
 '1시간20분(대용량배터리기준)',
 '20분',
 '40분',
 '40분',
 '',
 '40분',
 '45분',
 '50분',
 '42분',
 '1시간',
 '1시간',
 '20분',
 '1시간',
 '28분',
 '20분',
 '50분',
 '1시간10분',
 '',
 '1시간',
 '1시간',


In [177]:
# 사용시간 단위를 통일
def convertHourToMin(time):
    try:
        if'시간' in time:
            hour = time.split('시간')[0]
            if '분' in time:
                minute = time.split('시간')[-1].split('분')[0]
            else:
                minute = 0
        else:
            hour = 0
            minute = time.split('분')[0]
        return int(hour)*60 , minute
    except:
        pass

In [179]:
newUseTimeList = []
for time in useTimeList :
    value = convertHourToMin(time)
    newUseTimeList.append(value)

In [186]:
# 흡입력 단위 통일 1W = 1AW = 100pa
def convertPow(value):
    try:
        value = value.upper()
        if 'AW' in value or 'W' in value:
            result = value.replace('A','').replace('W','').replace(',','')
            result = int(result)
        elif 'PA' in value:
            result = value.replace('PA','').replace(',','')
            result = int(result)//100
        else:
            result = None
        
        return result
    except:
        return None

In [187]:
newSuctionList = []
for power in suctionPowList:
    value = convertPow(power)
    newSuctionList.append(value)

In [190]:
dfLast = pd.DataFrame()
dfLast['카테코리'] = CateList
dfLast['회사명'] = compList
dfLast['제품명'] = modelList
dfLast['가격'] = dfprodDanawa['최저가']
dfLast['사용시간'] = newUseTimeList
dfLast['흡입력'] = newSuctionList

In [192]:
dfLast.tail()

,카테코리,회사명,제품명,가격,사용시간,흡입력
2105,차량용청소기,미니센,무선 소형 청소기 MNC-100 노즐 세트,7000,"(0, )",NaN
2106,차량용청소기,미니센,무선 소형 청소기 MNC-100 먼지통,7000,"(0, )",NaN
2107,차량용청소기,프리라벨,2in1 무선 진공 청소기 LS-2000 헤파필터,3500,"(0, )",NaN
2108,침구청소기,포쉬,워시젯 V2 WJ200,219000,"(0, 25)",NaN
2109,침구청소기,업킵,MJ-BC779,89900,"(0, )",NaN


In [191]:
dfLast.to_excel('./data/무선청소기_최종.xlsx',index=False)

,카테코리,회사명,제품명,가격,사용시간,흡입력
0,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9S AX9884,873560,"(0, 30)",250.0
1,핸디스틱청소기,삼성전자,비스포크 제트 VS20B956AX,446320,"(60, 0)",220.0
2,핸디스틱청소기,샤오미,미홀 M22,151050,"(0, )",NaN
3,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9S AX9604,715570,"(0, 30)",250.0
4,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9S AX9988,1229240,"(60, 0)",280.0
...,...,...,...,...,...,...
2105,차량용청소기,미니센,무선 소형 청소기 MNC-100 노즐 세트,7000,"(0, )",NaN
2106,차량용청소기,미니센,무선 소형 청소기 MNC-100 먼지통,7000,"(0, )",NaN
2107,차량용청소기,프리라벨,2in1 무선 진공 청소기 LS-2000 헤파필터,3500,"(0, )",NaN
2108,침구청소기,포쉬,워시젯 V2 WJ200,219000,"(0, 25)",NaN
